# 定义模型结构的常用方法

1. 从 Module 类定义模型
2. 在 init 函数中定义网络层
3. 在 forward 函数中处理网络层连接

## 使用Module类定义网络层

### 定义网络层方法1：Sequential模块
在PyTorch中，Sequential是一个特殊的模块，它包含了其他模块，并按照它们被添加的顺序来执行它们的forward方法。<br>
这使得我们可以轻松地构建像堆栈一样的模型。以下是一个使用Sequential的例子：

In [64]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 10, 3),
            nn.ReLU(),
            nn.Linear(10, 2)
        )

In [66]:
import torch.nn as nn
from collections import OrderedDict

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(3, 10, 3)),
            ('relu', nn.ReLU()),
            ('fc', nn.Linear(10, 2)),
        ]))

### 定义网络层方法2：使用nn.ModuleList
nn.ModuleList是一个包含多个模块的列表。<br>
它可以包含任何类型的模块，并可以像普通Python列表一样进行索引。<br>
以下是一个示例：

In [61]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.ModuleList([nn.Conv2d(3, 10, 3), nn.ReLU(), nn.Linear(10, 2)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### 定义网络层方法3：使用nn.ModuleDict
nn.ModuleDict是一个包含多个模块的字典。<br>
它可以包含任何类型的模块，并可以像普通Python字典一样进行索引。<br>
以下是一个示例：

In [62]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.ModuleDict({
            'conv': nn.Conv2d(3, 10, 3),
            'relu': nn.ReLU(),
            'fc': nn.Linear(10, 2)
        })

    def forward(self, x):
        x = self.layers['conv'](x)
        x = self.layers['relu'](x)
        x = self.layers['fc'](x)
        return x

### 定义网络层方法4：一般写法

In [74]:
import torch.nn as nn
class MyNet(nn.Module):
    def __init__(self, inputdim, hiddendim, outputdim):
        super(MyNet, self).__init__()
        self.Linear1 = nn.Linear(inputdim, hiddendim)
        self.Linear2 = nn.Linear(hiddendim, outputdim)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [75]:
import torch.nn as nn
class MyNet(nn.Module):
    def __init__(self, inputdim, hiddendim, outputdim):
        super(MyNet, self).__init__()
        # ---- 与上面的写法等价
        self.add_module("Linear1", nn.Linear(inputdim, hiddendim))
        self.add_module("Linear2", nn.Linear(hiddendim, outputdim))
        self.add_module("criterion", nn.CrossEntropyLoss())
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### children()
所有通过 `Module` 类定义的模型，都可以使用 `children()` 获得各层的信息：

In [76]:
model = MyNet(2, 3, 2)
for x in model.children():
    print(x)

Linear(in_features=2, out_features=3, bias=True)
Linear(in_features=3, out_features=2, bias=True)
CrossEntropyLoss()


### named_children()
通过 `named_children` 可以获得各层的名字信息：

In [77]:
for x in model.named_children():
    print(x)

('Linear1', Linear(in_features=2, out_features=3, bias=True))
('Linear2', Linear(in_features=3, out_features=2, bias=True))
('criterion', CrossEntropyLoss())


### modules()
或者，通过 `modules` 获得更详细信息：

In [78]:
for m in model.modules():
    print(m)

MyNet(
  (Linear1): Linear(in_features=2, out_features=3, bias=True)
  (Linear2): Linear(in_features=3, out_features=2, bias=True)
  (criterion): CrossEntropyLoss()
)
Linear(in_features=2, out_features=3, bias=True)
Linear(in_features=3, out_features=2, bias=True)
CrossEntropyLoss()


## 为模型添加参数：Parameters

Parameters是Variable的子类。

### 为模型添加参数

**在init函数中定义网络层时，已经自动将网络层的参数传递给模型。**

### register_parameter()
也可以通过 `register_parameter(name, param)` 向模型添加。<br>
这也等价于 `self.p = Parameter(tensor)`。

### register_buffer(name, tensor)
如果需要在模型中保存状态，但不想作为模型参数，可以使用 `register_buffer(name, tensor)` 添加变量。

In [79]:
import torch
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, inputdim, hiddendim, outputdim):
        super(Net, self).__init__()
        # ----
        self.register_parameter('my_vector1', nn.Parameter(torch.randn(768, 3, 3)))
        self.register_parameter('my_vector2', nn.Parameter(torch.randn(1024, 768)))
        self.register_buffer('my_vector3', torch.tensor([2.0, 2.0]))

m = Net(2, 2, 2)
print(m.my_vector1.requires_grad) # 这是一个模型参数
print(m.my_vector3.requires_grad) # 这是一个普通变量

True
False


## 从模型中获取参数

### parameters() 列举参数

In [80]:
for p in m.parameters():
    print(p.shape)

torch.Size([768, 3, 3])
torch.Size([1024, 768])


### named_parameters() 列举参数

In [81]:
for name, p in m.named_parameters():
    print(name, ":", "/", p.shape)

my_vector1 : / torch.Size([768, 3, 3])
my_vector2 : / torch.Size([1024, 768])


### numel() 计算参数量

In [82]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'模型一共有 {count_parameters(m)} 个可训练参数！')

模型一共有 793344 个可训练参数！


### state_dict() 获取模型全部参数
**包括了模型参数（Parameters）和状态参数（Buffers）**

In [85]:
state = m.state_dict()
for p in state:
    print(state[p].shape)

torch.Size([768, 3, 3])
torch.Size([1024, 768])
torch.Size([2])


**查看权重：**

In [94]:
import torch.nn as nn
class MyNet(nn.Module):
    def __init__(self, inputdim, hiddendim, outputdim):
        super(MyNet, self).__init__()
        # ---- 与上面的写法等价
        self.add_module("Linear1", nn.Linear(inputdim, hiddendim))
        self.add_module("Linear2", nn.Linear(hiddendim, outputdim))
        self.add_module("criterion", nn.CrossEntropyLoss())

model = MyNet(2, 5, 2)
params = model.state_dict()
print(params['Linear1.weight'])
print(params['Linear1.bias'])

tensor([[-0.3595,  0.5266],
        [-0.1143,  0.0146],
        [-0.2233,  0.1463],
        [-0.2924, -0.5185],
        [-0.3401,  0.3198]])
tensor([-0.1163, -0.4208,  0.6642,  0.6232, -0.3497])


## 激活函数
每个激活函数都有两种形式：
- 类形式：在 torch.nn 模块中定义
- 函数形式：在 torch.nn.functional 模块中定义

因为激活函数通常都不会参与梯度计算，所以一般应当使用函数形式。

In [99]:
tanh = torch.nn.Tanh()
tanh(torch.tensor([1, 2, 3]))

tensor([0.7616, 0.9640, 0.9951])

**下面写法达到效果类似，却完全不会参与梯度计算：**

In [100]:
torch.nn.functional.tanh(torch.tensor([1, 2, 3]))

tensor([0.7616, 0.9640, 0.9951])

## L2正则化
PyTorch可以直接使用优化器中的 `weight_decay` 参数来指定权重衰减率，实现对权重 w 和 偏执 b 的L2正则化。

**有时需要指定优化器配置，仅对权重 w 做正则化，而不处理偏置 b，以防止过拟合。**

## Dropout 函数
Dropout同样有类形式和函数形式两种，与激活函数的情况类似，一般推荐用函数形式。
- Dropout：对 1 维数据进行 Dropout 处理
- Dropout2D：对 2 维数据进行 Dropout 处理
- Dropout3D：对 3 维数据进行 Dropout 处理

## 批量归一化

### 三种批量归一化处理
- BatchNorm1d: 处理2维或3维数据， `[N, D]`或`[N, D, L]`，N 批次，D 数据个数，L 数据长度
- BatchNorm2d：处理4维数据, `[N, C, H, W]`，C 通道数，H 高度，W 宽度
- BatchNorm3d：处理5维数据，`[N, C, D, H, W]`，D 深度

### 手动实现批量归一化算法示例